In [1]:
import sys

package_path: str = ".."
if package_path not in sys.path:
    sys.path.append(package_path)

In [3]:
import os
os.chdir("../..")

In [5]:
import numpy as np
from pathlib import Path
root = Path("/home/xssmgr/Desktop/sogang251128/analysis/processed_data/run=008/scan=001")
data = np.load(root / "run=0008_scan=0001_standard.npz")

In [6]:
for key, val in data.items():
    print(key, val.shape)

delay (81,)
pon (81, 514, 1030)
poff (81, 514, 1030)


In [11]:
from datetime import datetime
from pathlib import Path

import matplotlib.pyplot as plt
from pandas import DataFrame
import numpy as np
import numpy.typing as npt
import tifffile
from roi_rectangle import RoiRectangle

from CordaX.analyzer.core import DataAnalyzer
from CordaX.analyzer.draw_figure import (
    patch_rectangle,
    Visualizer
)
from CordaX.config import ConfigManager
from CordaX.config import ExpConfig
from CordaX.filesystem import get_run_scan_dir, make_run_scan_dir
from CordaX.gui.roi_core import RoiSelector
from CordaX.logger import Logger, setup_logger


ConfigManager.initialize("config.yaml")
config: ExpConfig = ConfigManager.load_config()
logger: Logger = setup_logger()
suffix: str = "standard"
run_nums = [66, 67, 73, 81]
logger.info(f"Data Analysing run={run_nums}")
scan_num: int = 1
now = datetime.now()
last_dir_name: str = now.strftime("%Y%m%d_%H%M%S")

processed_dir: Path = config.path.processed_dir
file_name: str = f"run={run_nums[0]:0>4}_scan={scan_num:0>4}"
if suffix:
    file_name += f"_{suffix}"
npz_file: Path = get_run_scan_dir(
    processed_dir, run_nums[0], scan_num, sub_path=file_name
).with_suffix(".npz")
logger.info(f"NPZ file: {npz_file}")

if not npz_file.exists():
    err_msg: str = f"The file {npz_file} does not exist."
    logger.error(err_msg)
    raise FileNotFoundError(err_msg)
# Initialize MeanDataProcessor
processor: DataAnalyzer = DataAnalyzer(npz_file)
poff_images: npt.NDArray = processor.poff_images
pon_images: npt.NDArray = processor.pon_images
# Select ROI using GUI

# TODO: Refactor the logic
plot_imgs = (poff_images + pon_images)
plot_imgs = (plot_imgs - np.min(plot_imgs)) / np.std(plot_imgs) * 1e3
if  not (roi := RoiSelector().select_roi(np.log1p(np.mean(plot_imgs, 0)))):
    err_msg: str = f"No ROI Rectangle Set for run={run_nums[0]}, scan={1}"
    logger.error(err_msg)
    raise ValueError(err_msg)

logger.info(f"ROI rectangle: {roi}")
roi_rect: RoiRectangle = RoiRectangle.from_tuple(roi)
fig, axs = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
fig.suptitle(str(run_nums))
for run_num in run_nums:

    logger.info(f"Run DataAnalyzer run={run_num:0>3} scan={scan_num:0>3}")
    file_name: str = f"run={run_num:0>4}_scan={scan_num:0>4}"
    if suffix:
        file_name += f"_{suffix}"
    npz_file: Path = get_run_scan_dir(
        processed_dir, run_num, scan_num, sub_path=file_name
    ).with_suffix(".npz")

    logger.info(f"NPZ file: {npz_file}")

    if not npz_file.exists():
        err_msg: str = f"The file {npz_file} does not exist."
        logger.error(err_msg)
        raise FileNotFoundError(err_msg)
    # Initialize MeanDataProcessor
    processor: DataAnalyzer = DataAnalyzer(npz_file)
        # Analyze data within the selected ROI
    data_df: DataFrame = processor.analyze_by_roi(roi_rect)

    delay = data_df.index
    poff_com_x = data_df["poff_com_x"]
    pon_com_x = data_df["pon_com_x"]
    poff_com_y = data_df["poff_com_y"]
    pon_com_y = data_df["pon_com_y"]

    # Calculate the difference between pon_com_x and poff_com_x
    com_x_difference = poff_com_x - pon_com_x

    # Calculate the difference between pon_com_y and poff_com_y
    com_y_difference = poff_com_y - pon_com_y


    # Plot com_x_difference
    axs[0].plot(
        delay,
        com_x_difference,
        label=f"{run_num}",
        marker=".",
    )
    axs[0].set_title("COM X Position Difference (poff - pon)")
    axs[0].set_ylabel("Position X Difference($Q_z$) [Å$^{-1}$]")

    # Plot com_y_difference
    axs[1].plot(
        delay,
        com_y_difference,
        label=f"{run_num}",
        marker=".",
    )
    axs[1].set_title("COM Y Position Difference (poff - pon)")
    axs[1].set_xlabel("Delay [ps]")
    axs[1].set_ylabel("Position Y Difference($Q_{xy}$) [Å$^{-1}$]")

    for ax in axs.flatten():
        ax.grid(True, linestyle='--', alpha=0.6)
        ax.legend()

    plt.tight_layout()

# Define save directory
# output_dir: Path = make_run_scan_dir(
#     config.path.output_dir, run_num, scan_num, sub_path=roi_name
# )
output_dir = Path(config.path.output_dir) / str(run_nums) / last_dir_name
output_dir.mkdir(parents=True, exist_ok=True)

# Slice images to ROI
roi_poff_images: npt.NDArray = roi_rect.slice(poff_images)
roi_pon_images: npt.NDArray = roi_rect.slice(pon_images)

# Save Images and Data
data_file: Path = output_dir / "data.csv"
data_df.to_csv(data_file)
logger.info(f"Saved CSV '{data_file}'")

# Save Figures
figures_to_save = {
    "delay-com_diff.png": fig,
}
for filename, fig in figures_to_save.items():
    file_path = output_dir / filename
    fig.savefig(file_path)
    logger.info(f"Saved PNG '{file_path}'")

logger.info(f"Run DataAnalyzer run={run_num:0>3} scan={scan_num:0>3} is Done.")
plt.close("all")


2025-11-29 17:04:39.725 | INFO     | __main__:<module>:28 - Data Analysing run=[66, 67, 73, 81]
2025-11-29 17:04:39.726 | INFO     | __main__:<module>:40 - NPZ file: /home/xssmgr/Desktop/sogang251128/analysis/processed_data/run=066/scan=001/run=0066_scan=0001_standard.npz
2025-11-29 17:04:44.662 | INFO     | __main__:<module>:60 - ROI rectangle: (85, 182, 343, 436)
2025-11-29 17:04:44.675 | INFO     | __main__:<module>:66 - Run DataAnalyzer run=066 scan=001
2025-11-29 17:04:44.676 | INFO     | __main__:<module>:74 - NPZ file: /home/xssmgr/Desktop/sogang251128/analysis/processed_data/run=066/scan=001/run=0066_scan=0001_standard.npz
2025-11-29 17:04:45.735 | INFO     | __main__:<module>:66 - Run DataAnalyzer run=067 scan=001
2025-11-29 17:04:45.736 | INFO     | __main__:<module>:74 - NPZ file: /home/xssmgr/Desktop/sogang251128/analysis/processed_data/run=067/scan=001/run=0067_scan=0001_standard.npz
2025-11-29 17:04:46.792 | INFO     | __main__:<module>:66 - Run DataAnalyzer run=073 scan=

In [12]:
import h5py
import hdf5plugin  # pylint: disable=unused-import

import pandas as pd

config = ConfigManager.load_config()
file = get_run_scan_dir(config.path.load_dir, 162, 1, sub_path="p0010.h5")
metadata_pd: pd.DataFrame = pd.read_hdf(file, key="metadata")
# print(metadata_pd)
print(metadata_pd.columns)
print(metadata_pd.index)
print(metadata_pd.shape)
metadata_pd

Index(['th_input', 'th_value', 'detector_eh1_jungfrau2_bkg_offset',
       'detector_eh1_jungfrau2_boardID', 'detector_eh1_jungfrau2_bunchid',
       'detector_eh1_jungfrau2_calibrated',
       'detector_eh1_jungfrau2_dark_bkg_substrated',
       'detector_eh1_jungfrau2_detecor_geometry',
       'detector_eh1_jungfrau2_device', 'detector_eh1_jungfrau2_dim1',
       ...
       'timestamp_info.THIRTY_HERTZ', 'timestamp_info.TWENTY_HERTZ',
       'timestamp_info.XFEL_BEAM', 'timestamp_info.XFEL_HX_BEAM',
       'timestamp_info.XFEL_SX_BEAM', 'timestamp_info.pulse_id',
       'timestamp_info.timestamp_nsec', 'timestamp_info.timestamp_offset',
       'timestamp_info.timestamp_sec', 'timestamp_info.timestamp_usec'],
      dtype='object', length=231)
Index([1717974299424, 1717974299441, 1717974299458, 1717974299474,
       1717974299491, 1717974299508, 1717974299524, 1717974299541,
       1717974299558, 1717974299574,
       ...
       1717974319276, 1717974319293, 1717974319309, 171797431932

,th_input,th_value,detector_eh1_jungfrau2_bkg_offset,detector_eh1_jungfrau2_boardID,detector_eh1_jungfrau2_bunchid,detector_eh1_jungfrau2_calibrated,detector_eh1_jungfrau2_dark_bkg_substrated,detector_eh1_jungfrau2_detecor_geometry,detector_eh1_jungfrau2_device,detector_eh1_jungfrau2_dim1,...,timestamp_info.THIRTY_HERTZ,timestamp_info.TWENTY_HERTZ,timestamp_info.XFEL_BEAM,timestamp_info.XFEL_HX_BEAM,timestamp_info.XFEL_SX_BEAM,timestamp_info.pulse_id,timestamp_info.timestamp_nsec,timestamp_info.timestamp_offset,timestamp_info.timestamp_sec,timestamp_info.timestamp_usec
timestamp,,,,,,,,,,,,,,,,,,,,,
1717974299424,18.6235,18.6239,-0.314614,0,1848406891374811035,True,False,normal,eh1:jungfrau2,512,...,True,False,True,True,False,56388,424860740,0.000056,1717974299,424804
1717974299441,18.6235,18.6239,-0.067718,0,1919901561229121435,True,False,normal,eh1:jungfrau2,512,...,False,True,True,True,False,56394,441506890,0.000056,1717974299,441450
1717974299458,18.6235,18.6239,-0.043325,0,1991959181036853147,True,False,normal,eh1:jungfrau2,512,...,True,False,True,True,False,56400,458153040,0.000056,1717974299,458096
1717974299474,18.6235,18.6239,0.137630,0,2063453850891163547,True,False,normal,eh1:jungfrau2,512,...,False,False,True,True,False,56406,474799190,0.000056,1717974299,474742
1717974299491,18.6235,18.6239,0.152752,0,2134948520745473947,True,False,normal,eh1:jungfrau2,512,...,True,True,True,True,False,56412,491576412,0.000057,1717974299,491520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1717974319359,18.6235,18.6239,0.074740,0,1569777459346773935,True,False,normal,eh1:jungfrau2,512,...,False,True,True,True,False,63558,359987270,0.000064,1717974319,359923
1717974319376,18.6235,18.6239,0.130375,0,1641272129201084335,True,False,normal,eh1:jungfrau2,512,...,True,False,True,True,False,63564,376633420,0.000064,1717974319,376569
1717974319393,18.6235,18.6239,0.075681,0,1713329749008816047,True,False,normal,eh1:jungfrau2,512,...,False,False,True,True,False,63570,393279570,0.000064,1717974319,393216


In [15]:
with h5py.File(file, "r") as hf:
    metadata_group = hf["metadata"]
    columns = metadata_group["axis0"]
    for key in metadata_group.keys():
        print(key, metadata_group[key].shape)

axis0 (231,)
axis1 (1200,)
block0_items (30,)
block0_values (1200, 30)
block1_items (79,)
block1_values (1200, 79)
block2_items (58,)
block2_values (1200, 58)
block3_items (64,)
block3_values (1,)


In [3]:
config = ConfigManager.load_config()
npz_root: Path = config.path.processed_dir
run: int = 162
print("RUN:", run)
npz_file: Path = get_run_scan_dir(
    npz_root, run, 1, sub_path=f"run={run:04}_scan=0001.npz"
)
data = np.load(npz_file)
data

RUN: 162


NpzFile 'Y:\\240608_FXS\\analysis_data\\processed_data\\run=162\\scan=001\\run=0162_scan=0001.npz' with keys: pon, poff, delay

In [4]:
print("Delay:", data["delay"])
print("poff", data["poff"])

Delay: [18.5554 18.5637 18.5713 18.5789 18.5862 18.5938 18.6012 18.6087 18.6162
 18.6239 18.6314 18.6388 18.6462 18.6537 18.6613 18.6689 18.6762 18.6837
 18.6911 18.6986 18.7063 18.7138 18.7213 18.7288 18.7363 18.7439 18.7513
 18.7588 18.7663 18.7737 18.7812 18.7888 18.7963 18.8037 18.8112 18.8189
 18.8264 18.8338 18.8413 18.8488 18.8563 18.8637 18.8712 18.8787 18.8863
 18.8937 18.9012 18.9087 18.9162 18.9238 18.9313 18.9387 18.9613 18.9689
 18.9764 18.9913 18.9987 19.0062 19.0137 19.0213 19.0287 19.0364 19.0437
 19.0513 19.0589 19.0664 19.0739 19.0814 19.0888 19.0962 19.1037 19.1114
 19.1188 19.1264 19.1338 19.1412 19.1489 19.1562]
poff [[[0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  ...
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0